# <font color='blue'>ALPAR - Governo Digital - Processo Inteligente</font>

# <font color='red'>ETL para o BI</font>

## 1.0 import

In [1]:
# !pip install pandas
# !pip install mysql-connector-python

In [2]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode# import numpy as np
# import copy
# import csv
# import codecs
# import time
# from os.path import expanduser



# print("pandas versão", pd.__version__)
# print("numpy versão", np.__version__)
# print("csv versão", csv.__version__)

# pd.options.display.max_rows = 2000
# pd.options.display.width = 120
# pd.options.display.max_colwidth = 100

## <font color='black'>2.0 load datasets</font>

In [3]:
My_host = 'localhost'
My_db = 'bd_fontes'
My_user = 'gd'
My_pw = 'Alpar@123'

In [4]:
sql_form = (
"SELECT * "
"FROM form"
)
sql_tasks = (
"SELECT "
"Protocolo as 'protocolo', "
"Entidade as 'entidade', "
"`Data e Hora de criação` as 'data_hora_criacao' "
"FROM tasks"
)

In [5]:
try:
    connection = mysql.connector.connect(host = My_host, database = My_db, user = My_user, password = My_pw)
    
    df_form = pd.read_sql(sql_form, con=connection)
    df_tasks = pd.read_sql(sql_tasks, con=connection)
    
except mysql.connector.Error as error:
    print("Failed to read record from MySQL table {}".format(error))

finally:
    if (connection.is_connected()):
        connection.close()
        print(f'{df_form.shape[0]} registros lidos em {df_form.shape[1]} colunas')
        print(f'{df_tasks.shape[0]} registros lidos em {df_tasks.shape[1]} colunas')

        print("MySQL connection is closed")

2034 registros lidos em 7 colunas
619 registros lidos em 3 colunas
MySQL connection is closed


## <font color='black'>3.0 prepara tabelas auxiliares</font>

### 3.1 prepara tabela auxiliar tasks (Z_aux_entidade)

In [6]:
df_aux_entidade = df_tasks[['protocolo', 'entidade']].drop_duplicates()
# df_aux_entidade

### 3.2 prepara tabela BASE (Z_aux_BASE)

In [7]:
df_aux_BASE = df_form.copy()
df_aux_BASE.columns = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 'campo_relacionado']

In [8]:
# transforma as strings atributo e campo relacionado em lower case
df_aux_BASE[['atributo', 'campo_relacionado']] = df_aux_BASE[['atributo', 'campo_relacionado']].apply(lambda x: x.str.lower())
# df_aux_BASE

In [9]:
# faz um merge da dimensão df_aux_BASE com a df_auxentidade para adicionar a coluna entidade
# se não existir o protocolo na tasks não considera a linha -- isso é um erro de integridade da base de dados
df_aux_BASE = df_aux_BASE.merge(df_aux_entidade, on='protocolo')

In [10]:
# cria 3 novas colunas vazias
df_aux_BASE['MET_atributo'] = np.nan
df_aux_BASE['MET_valor'] = np.nan
df_aux_BASE['MET_tipo'] = np.nan

# cria uma coluna como PK
df_aux_BASE['PK'] = df_aux_BASE['entidade'] + '_' + df_aux_BASE['servico'] + '_' + \
                    df_aux_BASE['atributo'] + '_' + df_aux_BASE['protocolo']

# cria uma coluna com o tipo de registro
df_aux_BASE['TIPO_REG'] = 'BASE'

### 3.3 prepara tabela MET (Z_aux_MET)

In [11]:
def Enumerico(s, tipo):
    try:
        v = np.float64(s) if tipo == 'DOUBLE' else np.int64(s)
    except:
        return False
    
    return True

In [12]:
def GeraCodDimMet(d, m):
    d_dim = {'STRING_DIM': 10, 'BOOLEAN_DIM': 20, 'INTEGER_DIM' :30, 'DOUBLE_DIM': 40, 'ID_DIM': 50}
    d_met = {'INTEGER_MET': 1, 'DOUBLE_MET': 2, 'NULL_MET': 3}

    v_dim = d_dim[d] if d in d_dim else 0
    v_met = d_met[m] if m in d_met else 0
    
    return v_dim + v_met

In [13]:
# traramento do tipo INTEGER ou DOUBLE

# dataset de métricas - usa somente tipo "INTEGER" ou "DOUBLE"
cols = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 'campo_relacionado', 'entidade']
df_aux_MET = df_aux_BASE.loc[(df_aux_BASE['tipo'] == 'INTEGER') | (df_aux_BASE['tipo'] == 'DOUBLE'), cols].copy()

# os valores estão com formato: ponto para separador de milhar e vírgula para fração
# tira os pontos separador de milhar e substitui vírgula por ponto
df_aux_MET['valor'] = df_aux_MET['valor'].apply(lambda x: x.replace('.', '').replace(',', '.'))

# alimenta uma coluna indicadora que mostra se o valor corresponde ou não ao tipo
df_aux_MET['numerico'] = df_aux_MET.apply(lambda x: Enumerico(x['valor'], x['tipo']), axis=1)

# cria um dataset de linhas cuja o valor não corresponde ao tipo
df_aux_MET_Err = df_aux_MET[df_aux_MET['numerico'] == False].drop('numerico', axis=1)

# cria um dataset de linhas de métricas
df_aux_MET = df_aux_MET[~df_aux_MET['numerico'] == False].drop('numerico', axis=1)

# identifica que esse é um dataset de métricas
df_aux_MET['TIPO_REG'] = 'MET'

# exclui linhas que não seja uma métrica explícita
df_aux_MET = df_aux_MET[~(df_aux_MET['campo_relacionado'] == 'null')]

# cria a coluna FK para relacionar com o dataset BASE
df_aux_MET['FK'] = df_aux_MET['entidade'] + '_' + df_aux_MET['servico'] + '_' + \
                   df_aux_MET['campo_relacionado'] + '_' + df_aux_MET['protocolo']

# troca o nome de algumas colunas
dc = {'atributo': 'MET_atributo', 'campo_relacionado': 'atributo', 'valor': 'MET_valor', 'tipo': 'MET_tipo'}
df_aux_MET.rename(columns=dc, inplace = True)

# faz um merge com o dataset BASE 
df_aux_MET = df_aux_MET.merge(df_aux_BASE[['PK', 'valor', 'tipo']], left_on='FK', right_on='PK')
df_aux_MET.drop(['PK', 'FK'], axis=1, inplace=True)


### 3.4 prepara tabela DIM (Z_aux_DIM)

In [14]:
# escolee as colunas que farão a cancatenação de dataframes
cols = ['atributo', 'nome', 'valor', 'protocolo', 'servico', 'tipo', 
        'entidade', 'MET_atributo', 'MET_valor', 'MET_tipo', 'TIPO_REG']
df_aux_DIM = df_aux_BASE[cols].copy()
df_aux_DIM = pd.concat([df_aux_DIM, df_aux_MET])

### 3.5 prepara tabela de decisão (Z_aux_decisao)

In [15]:
mtx =  [('STRING_DIM',  'INTEGER_MET', 'N', 'N', 'S', 'S'),
        ('STRING_DIM',  'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('STRING_DIM',  'NULL_MET',    'S', 'S', 'N', 'N'),
        
        ('BOOLEAN_DIM', 'INTEGER_MET', 'S', 'N', 'N', 'N'),
        ('BOOLEAN_DIM', 'DOUBLE_MET',  'S', 'N', 'N', 'N'),
        ('BOOLEAN_DIM', 'NULL_MET',    'S', 'N', 'N', 'N'),
        
        ('INTEGER_DIM', 'INTEGER_MET', 'S', 'S', 'S', 'S'),
        ('INTEGER_DIM', 'DOUBLE_MET',  'N', 'N', 'N', 'N'),
        ('INTEGER_DIM', 'NULL_MET',    'S', 'S', 'N', 'N'),
        
        ('DOUBLE_MET',  'INTEGER_MET', 'N', 'N', 'N', 'N'),
        ('DOUBLE_MET',  'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('DOUBLE_MET',  'NULL_MET',    'N', 'N', 'N', 'N'),
        
        ('ID_DIM',      'INTEGER_MET', 'N', 'N', 'S', 'S'),
        ('ID_DIM',      'DOUBLE_MET',  'N', 'N', 'S', 'S'),
        ('ID_DIM',      'NULL_MET',    'S', 'S', 'N', 'N')
       ]
cols_mtx = ['t_dim', 't_met', 'Count', 'Distinct Count', 'Sum', 'Average']

df_aux_dec = pd.DataFrame(mtx, columns=cols_mtx)
df_aux_dec['Cod_Decisao'] = df_aux_dec.apply(lambda x:
                                    GeraCodDimMet(x['t_dim'], x['t_met']), axis=1)

# df_aux_dec

### 3.6 ponte de decisão da operação (Z_pto_decisao_operacao)

In [16]:
# faz uma cópia da tabela de decisão com as colunas necessárias
cols = ['Cod_Decisao'] + cols_mtx[2:]
df_aux_pto = df_aux_dec[cols].copy()

# faz o unpivot
df_aux_pto = pd.melt(df_aux_pto, id_vars=['Cod_Decisao'], 
                 value_vars=['Count', 'Distinct Count', 'Sum', 'Average'],
                 var_name='Atributo')
df_aux_pto = df_aux_pto[df_aux_pto['value'] == 'S'].drop('value', axis=1)

# trasnforma a coluna Atributo em uma coluna do tipo "category'
df_aux_pto['Atributo'] = df_aux_pto['Atributo'].astype('category')

# cria uma nova coluna que será a coluna chave primária da dimensão Operação
df_aux_pto['FK_ZN_dim_OPERACAO'] = df_aux_pto['Atributo'].cat.codes.astype('int64') + 1

# cria a tabela df_pto_decisao_operacao necessária ao modelo dimensional
df_pto_decisao_operacao = df_aux_pto[['Cod_Decisao', 'FK_ZN_dim_OPERACAO']]
df_pto_decisao_operacao.columns = ['FK_dim_DECISAO', 'FK_ZN_dim_OPERACAO']
# df_pto_decisao_operacao

### 3.7 tabela auxiliar das Datas das Solicitações (Z_aux_DataSolicitacao)

In [ ]:
# cria uma coluna só de datas
df_tasks['data_criacao'] = pd.to_datetime(df_tasks['data_hora_criacao']).dt.date

# pega a menor data de cada número de protocolo
df_aux_DataSolicitacao = df_tasks[['protocolo', 'data_criacao']].groupby('protocolo').agg('min').reset_index()
# df_aux_DataSolicitacao

## <font color='black'>4.0 monta tabelas dimensões</font>

### 4.1 monta Z_dim_DIM

In [ ]:
df_dim_dim = df_aux_DIM[['atributo', 'nome']].drop_duplicates().reset_index(drop=True).reset_index()
df_dim_dim.columns = ['PK_Z_dim_DIM', 'dimensao', 'nome']
df_dim_dim['PK_Z_dim_DIM'] += 1
# df_dim_dim

### 4.2 monta Z_dim_SERVICO

In [ ]:
df_dim_servico = df_aux_DIM['servico'].drop_duplicates().reset_index(drop=True).reset_index()
df_dim_servico.columns = ['PK_Z_dim_SERVICO', 'servico']
df_dim_servico['PK_Z_dim_SERVICO'] += 1
# df_dim_servico


### 4.3 monta Z_dim_MET

In [ ]:
df_dim_met = df_aux_DIM.loc[~df_aux_DIM['MET_atributo'].isna(), 'MET_atributo'].drop_duplicates()
df_dim_met = df_dim_met.reset_index(drop=True).reset_index()
df_dim_met.columns = ['PK_Z_dim_MET', 'metrica']
df_dim_met['PK_Z_dim_MET'] += 1
# df_dim_met


### 4.4 monta Z_dim_OPERACAO

In [ ]:
df_ZN_dim_OPERACAO = df_aux_pto[['FK_ZN_dim_OPERACAO', 'Atributo']].drop_duplicates()
df_ZN_dim_OPERACAO.columns = ['PK_Z_dim_OPERACAO', 'operacao']
# df_ZN_dim_OPERACAO

### 4.5 monta Z_dim_DECISÃO

In [ ]:
df_dim_dec = df_aux_dec[['Cod_Decisao', 't_dim', 't_met']].copy().rename(columns={'Cod_Decisao': 'PK_Z_dim_DECISAO'})
# df_dim_dec

### 4.6 monta Z_fat_BETA

In [ ]:
# copia o df auxiliar
df_fat_B = df_aux_DIM.copy()

# faz merge com dim_DIM
df_fat_B = df_fat_B.merge(df_dim_dim, 
                          how='left', 
                          left_on=['atributo', 'nome'], 
                          right_on=['dimensao', 'nome']).drop(['atributo', 'nome', 'dimensao'], axis=1)
# faz merge com dim_SERVICO
df_fat_B = df_fat_B.merge(df_dim_servico, 
                          how='left', 
                          left_on='servico', 
                          right_on='servico').drop('servico', axis=1)
# faz merge com dim_MET
df_fat_B = df_fat_B.merge(df_dim_met, 
                          how='left', 
                          left_on='MET_atributo', 
                          right_on='metrica').drop(['MET_atributo', 'metrica'], axis=1)
# faz merge com Z_aux_DataSolicitacao
df_fat_B = df_fat_B.merge(df_aux_DataSolicitacao, 
                          how='left', 
                          on='protocolo') #.drop(['MET_atributo', 'metrica'], axis=1)

# cria a coluna indicadora de NULL_MET
df_fat_B['reg_nativo'] = df_fat_B['MET_tipo'].apply(lambda x: 1 if pd.isna(x) else 0)

# passa as colunas de tipo para a string indicadora de NULL
df_fat_B.loc[df_fat_B['MET_tipo'].isna(), 'MET_tipo'] = 'NULL_MET'
df_fat_B.loc[df_fat_B['tipo'].isna(), 'tipo'] = 'NULL_DIM'


# substitui valores da coluna tipo
subs_dim = {'STRING':'STRING_DIM', 
            'INTEGER': 'INTEGER_DIM', 
            'BOOLEAN':'BOOLEAN_DIM', 
            'DOUBLE': 'DOUBLE_DIM',
            'ID':'ID_DIM'}
subs_met = {'INTEGER': 'INTEGER_MET', 
            'DOUBLE': 'DOUBLE_MET'}

df_fat_B['tipo'].replace(subs_dim, inplace=True)
df_fat_B['MET_tipo'].replace(subs_met, inplace=True)

# acha o código para a tabela de decisão da dupla de tipos como FK para a PK de "df_aux_dec"
df_fat_B['FK_dim_DECISAO'] = df_fat_B.apply(lambda x:
                                    GeraCodDimMet(x['tipo'], x['MET_tipo']), axis=1)

# exclui colunas já usadas
df_fat_B.drop(['tipo', 'MET_tipo'], axis=1, inplace=True)

# acerta os nomes das colunas
dc = {'PK_Z_dim_DIM': 'FK_Z_dim_DIM', 
      'PK_Z_dim_SERVICO': 'FK_Z_dim_SERVICO', 
      'PK_Z_dim_MET': 'FK_Z_dim_MET', 
      'data_criacao': 'FK_Data_Criacao'}
df_fat_B.rename(columns=dc, inplace = True)

# acerta os tipos das colunas
df_fat_B['FK_Z_dim_MET'] = df_fat_B['FK_Z_dim_MET'].astype('Int64')
df_fat_B['MET_valor'] = df_fat_B['MET_valor'].astype('float64')
df_fat_B['FK_Data_Criacao'] = df_fat_B['FK_Data_Criacao'].apply(pd.to_datetime)

# df_fat_B.head()

# grava no MySQL

In [ ]:
# monta as strings de conexão
def StrConn(sit):
    in_oper   = {'My_host': 'localhost', 'My_db': 'bd_fontes',       'My_user': 'gd', 'My_pw': 'Alpar@123'}
    out_oper  = {'My_host': 'localhost', 'My_db': 'bd_dw',           'My_user': 'gd', 'My_pw': 'Alpar@123'}
    in_teste  = {'My_host': 'localhost', 'My_db': 'bd_teste_fontes', 'My_user': 'gd', 'My_pw': 'Alpar@123'}
    out_teste = {'My_host': 'localhost', 'My_db': 'bd_teste_dw',     'My_user': 'gd', 'My_pw': 'Alpar@123'}
    
    # se True é teste
    if sit:
        rt_in = in_teste
        rt_out = out_teste
    else:
        rt_in = in_oper
        rt_out = out_oper

    return rt_in, rt_out

In [ ]:
def Load_BD(strAut):

    # import the module
    # create sqlalchemy engine
    
    SQLengine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                           .format(user = strAut['My_user'],
                                   pw   = strAut['My_pw'],
                                   host = strAut['My_host'],
                                   db   = strAut['My_db']),
                                   pool_recycle=3600)
    dbConn = SQLengine.connect()

    try:
        df_dim_Acao.to_sql('dim_acoes', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Categoria.to_sql('dim_categorias_servicos', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Encaminhamento.to_sql('dim_encaminhamento', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Entidade.to_sql('dim_entidades', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_GrupoResponsavel.to_sql('dim_grupo_responsavel', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Grupo.to_sql('dim_grupos_usuarios', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_MotivoCanc.to_sql('dim_motivos_canc', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Servicos.to_sql('dim_servicos', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_StatusExt.to_sql('dim_status_ext', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Usuarios.to_sql('dim_usuarios', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Situacao.to_sql('dim_situacao', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_SLA.to_sql('dim_sla', con=dbConn, if_exists='replace', index=False, chunksize = 1000)
        df_dim_Endereco.to_sql('dim_endereco', con=dbConn, if_exists='replace', index=False, chunksize = 1000)

        df_fat_tasks.to_sql('fat_tasks', con=dbConn, if_exists='replace', index=False, chunksize = 1000)

    except ValueError as vx:
        print('ERROR -', vx)

    except Exception as ex:
        print('EXCEPTION -', ex)

    else:
        print('Tabelas criadas com sucesso');  

    finally:
        dbConn.close()

# TESTE